In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
ALL_EVALUATIONS="tests/evaluation_13_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240117.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"

E5_RUN="tests/e5_docs_000_999_50_sampled_queries_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_999_50_sampled_queries_run.txt"
SPLADE_MT5_RUN="tests/splade_mt5_docs_000_099_50_sampled_queries_run.txt"

BM25_1M_RUN="tests/bm25_1M_docs_000_999_sampled_50_queries_run.txt"
BM25_1M_MASSIVEWEB_RUN="tests/bm25_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

COLBERTX_1M_MASSIVEWEB_RUN="tests/colbertx_all_retrieval_1M_dataset_20240105_run.txt"

E5_COLBERTX_RRF_RUN="tests/colbertx_e5_RRF_000_999_sampled_50_queries_run.txt"
E5_SPLADE_RRF_RUN="tests/splade_e5_RRF_000_999_sampled_50_queries_run.txt"

In [4]:
DATASET_1M_FIELDS=['bm25_1M_massiveweb', 'colbertx_1M_massiveweb']

In [5]:
def prepare_run_format(results_filename, add_index=True, score_field='score', invert_score=False, retriever_name="retriever"):
    
    results_df = pd.read_csv(results_filename, sep='\t')
    
    display(results_df.head())
    
    results_grouped_df = results_df.groupby("query_id")['passage_id'].count()
    
    how_many_passages_per_question = results_grouped_df.iloc[0]

    if invert_score:
        print("Inverting score values in '{}' field...".format(score_field))
        
        results_df['score_inv'] = 1 - results_df[score_field]
        score_field = 'score_inv' 
    
    run_df = results_df.sort_values(["query_id", score_field], ascending=[True, False])
    
    run_df['Q0'] = "Q0"
    run_df['comment'] = retriever_name
    
    if add_index:
        run_df['index'] = np.tile(list(range(1, how_many_passages_per_question + 1)), results_grouped_df.shape[0])
    
    output_filename = "{}_run.txt".format(os.path.splitext(results_filename)[0])
    
    print("output_filename={}".format(output_filename))
    
    run_df[['query_id', 'Q0', "passage_id", "index", score_field, "comment"]].to_csv(output_filename, header=None, index=False, sep=" ")
          
    return output_filename, run_df

In [6]:
retrievers_results = []
retrievers_1M_results = []

### Format GPT-4 evaluations to qrels format

In [7]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))
qrels_1M_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_1M_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [8]:
if os.path.exists(qrels_filename) == False:
    eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

    eval_df

    eval_df.drop_duplicates("query_id")[['query_id', 'query']]

    #### Check scores distribution on the current qrels

    np.unique(eval_df['score'].to_numpy(), return_counts=True)

    for group_name, group_df in eval_df.groupby('query_id'):
        print("\nquery_id={}".format(group_name))

        print(np.unique(group_df['score'].to_numpy(), return_counts=True))

    eval_df.groupby('query_id').count()

    eval_df.groupby('query_id').count()['query'].describe()

    #### Prepare the qrels format for trec_eval tools

    eval_df['0'] = '0'

    print("\n\n")
    print(qrels_filename)
    print(qrels_1M_filename)

    eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")
    
    
    #### Select 1M dataset evaluations
    
    eval_df[eval_df[DATASET_1M_FIELDS].sum(axis=1) > 0].sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_1M_filename, header=None, index=False, sep=" ")
    
else:
    print("QRELS files ({}) already created...".format(qrels_filename))


query_id=1
(array([0, 1, 2, 3]), array([14, 37, 15, 10]))

query_id=2
(array([0, 1, 2, 3]), array([78,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([50,  9, 11,  5]))

query_id=11
(array([0, 1, 2, 3]), array([39, 15,  8,  2]))

query_id=13
(array([0, 1, 2, 3]), array([30, 13, 20,  4]))

query_id=15
(array([0, 1, 2, 3]), array([20, 17,  3, 17]))

query_id=17
(array([0, 1, 2, 3]), array([19,  7,  1, 36]))

query_id=20
(array([0, 1, 2, 3]), array([15, 46, 21,  6]))

query_id=21
(array([0, 1, 2, 3]), array([44, 18,  5, 17]))

query_id=22
(array([0, 1, 2, 3]), array([48, 11,  2,  6]))

query_id=24
(array([0, 1, 2, 3]), array([49, 25, 11,  2]))

query_id=26
(array([0, 1, 2, 3]), array([13,  9, 12, 32]))

query_id=28
(array([0, 1, 2, 3]), array([50, 11, 13,  4]))

query_id=47
(array([0, 1, 2]), array([46, 24, 22]))

query_id=49
(array([0, 1, 2, 3]), array([23, 25, 20,  1]))

query_id=51
(array([0, 1, 2, 3]), array([26, 20, 11,  1]))

query_id=54
(array([0, 1, 2, 3]), array([ 1, 12, 

## 10M dataset

### Compute BM25 nDCG@10

In [9]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [10]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [11]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4527'}]

### Compute BM25 + E5 nDCG@10

In [12]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [13]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [14]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [15]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [16]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [17]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [18]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [19]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [20]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [21]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [22]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [23]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE + mT5 nDCG@10

In [24]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_MT5_RUN}

In [25]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR + mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + ColBERT-X RRF nDCG@10

In [26]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_COLBERTX_RRF_RUN}

In [27]:
retrievers_results.append({'retriever':'E5 + ColBERT-X mMARCO_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + SPLADE RRF nDCG@10

In [28]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_SPLADE_RRF_RUN}

In [29]:
retrievers_results.append({'retriever':'E5 + SPLADEv2_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M nDCG@10

In [30]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_RUN}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M massiveweb filtered nDCG@10

In [31]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X massiveweb filtered nDCG@10

In [32]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [33]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4527
1,BM25+E5,0.6450
2,BM25+mT5,0.7209
3,SPLADEv2_pt-BR,0.5868
4,ColBERT-X_mMARCO_pt-BR,0.6357
5,E5,0.5638
6,E5 base,0.4030
7,SPLADEv2_pt-BR + mT5,0.7067
8,E5 + ColBERT-X mMARCO_pt-BR RRF,0.6471
9,E5 + SPLADEv2_pt-BR RRF,0.5081


## 1M dataset

### Compute BM25 1M massiveweb filtered nDCG@10

In [ ]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X massiveweb filtered nDCG@10

In [ ]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_1M_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [ ]:
pd.DataFrame(retrievers_1M_results)